### LSE Data Analytics Online Career Accelerator

# DA201: Data Analytics using Python

## Assignment: Diagnostic Analysis using Python

This analysis is related to the appointments of National Health Service (NHS)

This jupyter notebook contains the major working procedures of handling the real datasets.


###  GitHub repository (Elaine Wong)
 > https://ElaineWong-HR/github.io/Wong_Elaine_DA201_Assignment/

## Import libraries

In [1]:
# Import the  libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.express as px 
import plotly.graph_objects as go
import plotly.offline
import plotly.graph_objs as go
from pyecharts import options as opts
from pyecharts.charts import Sankey

# Optional - Ignore warnings.
import warnings
warnings.filterwarnings('ignore')

## data cleansing


### Handle dataset 1: actual_duration

In [2]:
# Import and sense-check the actual_duration.csv data set as ad.
ad = pd.read_csv('actual_duration.csv')

# View the DataFrame.
ad.head()

,sub_icb_location_code,sub_icb_location_ons_code,sub_icb_location_name,icb_ons_code,region_ons_code,appointment_date,actual_duration,count_of_appointments
0,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01-Dec-21,31-60 Minutes,364
1,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01-Dec-21,21-30 Minutes,619
2,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01-Dec-21,6-10 Minutes,1698
3,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01-Dec-21,Unknown / Data Quality,1277
4,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01-Dec-21,16-20 Minutes,730


In [3]:
# Determine the metadata of the data set.
ad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137793 entries, 0 to 137792
Data columns (total 8 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   sub_icb_location_code      137793 non-null  object
 1   sub_icb_location_ons_code  137793 non-null  object
 2   sub_icb_location_name      137793 non-null  object
 3   icb_ons_code               137793 non-null  object
 4   region_ons_code            137793 non-null  object
 5   appointment_date           137793 non-null  object
 6   actual_duration            137793 non-null  object
 7   count_of_appointments      137793 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 8.4+ MB


In [4]:
# Convert the appointment date to datetime type
ad['appointment_date'] = pd.to_datetime(ad['appointment_date'])

# Determine the record period of ad dataframe
ad['appointment_date'].agg(['min','max'])

min   2021-12-01
max   2022-06-30
Name: appointment_date, dtype: datetime64[ns]

In [106]:
# Create some additional columns of dates in ad DataFrame

month_dict={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May',6:'Jun',
            7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

season_dict={1:'Winter',2:'Winter',3:'Winter',
             4:'Spring',5:'Spring',6:'Spring',
             7:'Summer',8:'Summer',9:'Summer',
             10:'Autumn',11:'Autumn',12:'Autumn'}

region_dict={'E40000003':'London', 'E40000005':'South East',
             'E40000006':'South West', 'E40000007':'East of England', 
             'E40000010':'North West', 'E40000011':'Midlands', 'E40000012':'North East and Yorkshire'}

ad['appointment_month'] = ad['appointment_date'].dt.to_period('M')
ad['year']=ad['appointment_date'].dt.year
ad['quarter']=ad['appointment_date'].dt.quarter
ad['month'] = ad['appointment_date'].dt.month
ad['month_year']=ad['month'].map(month_dict).astype(str)+'-'+ad.year.astype(str)
ad['season']=ad['month'].map(season_dict)

ad['weekday']=ad['appointment_date'].dt.day_name()

ad['region']=ad['region_ons_code'].map(region_dict)

ad.head()

,sub_icb_location_code,sub_icb_location_ons_code,sub_icb_location_name,icb_ons_code,region_ons_code,appointment_date,actual_duration,count_of_appointments,appointment_month,year,quarter,month,month_year,season,weekday,region
0,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,2021-12-01,31-60 Minutes,364,2021-12,2021,4,12,Dec-2021,Autumn,Wednesday,North East and Yorkshire
1,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,2021-12-01,21-30 Minutes,619,2021-12,2021,4,12,Dec-2021,Autumn,Wednesday,North East and Yorkshire
2,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,2021-12-01,6-10 Minutes,1698,2021-12,2021,4,12,Dec-2021,Autumn,Wednesday,North East and Yorkshire
3,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,2021-12-01,Unknown / Data Quality,1277,2021-12,2021,4,12,Dec-2021,Autumn,Wednesday,North East and Yorkshire
4,00L,E38000130,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,2021-12-01,16-20 Minutes,730,2021-12,2021,4,12,Dec-2021,Autumn,Wednesday,North East and Yorkshire


In [6]:
# Determine the number of unique elements of the data set.
ad.nunique()

sub_icb_location_code         106
sub_icb_location_ons_code     106
sub_icb_location_name         106
icb_ons_code                   42
region_ons_code                 7
appointment_date              212
actual_duration                 7
count_of_appointments        7611
appointment_month               7
year                            2
quarter                         3
month                           7
month_year                      7
season                          3
weekday                         7
region                          7
dtype: int64

In [7]:
# Determine whether there are missing values.
ad.isnull().sum()

sub_icb_location_code        0
sub_icb_location_ons_code    0
sub_icb_location_name        0
icb_ons_code                 0
region_ons_code              0
appointment_date             0
actual_duration              0
count_of_appointments        0
appointment_month            0
year                         0
quarter                      0
month                        0
month_year                   0
season                       0
weekday                      0
region                       0
dtype: int64

In [8]:
# Determine whether there are duplicated records.
ad.duplicated().sum()

0

In [9]:
ad.describe()

,count_of_appointments,year,quarter,month
count,137793.000000,137793.000000,137793.000000,137793.000000
mean,1219.080011,2021.858433,1.848766,4.700507
std,1546.902956,0.348607,0.988825,3.355861
min,1.000000,2021.000000,1.000000,1.000000
25%,194.000000,2022.000000,1.000000,2.000000
50%,696.000000,2022.000000,2.000000,4.000000
75%,1621.000000,2022.000000,2.000000,6.000000
max,15400.000000,2022.000000,4.000000,12.000000


In [10]:
# Determine the total actualized appointment in ad dataframe
ad['count_of_appointments'].sum()

167980692

## Handle dataset 2: appointments_regional

In [11]:
# Import and sense-check the appointments_regional.csv data set as ar.
ar = pd.read_csv('appointments_regional.csv')

# View the DataFrame.
ar.head()

,icb_ons_code,appointment_month,appointment_status,hcp_type,appointment_mode,time_between_book_and_appointment,count_of_appointments
0,E54000034,2020-01,Attended,GP,Face-to-Face,1 Day,8107
1,E54000034,2020-01,Attended,GP,Face-to-Face,15 to 21 Days,6791
2,E54000034,2020-01,Attended,GP,Face-to-Face,2 to 7 Days,20686
3,E54000034,2020-01,Attended,GP,Face-to-Face,22 to 28 Days,4268
4,E54000034,2020-01,Attended,GP,Face-to-Face,8 to 14 Days,11971


In [12]:
# Determine the metadata of the data set.
ar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596821 entries, 0 to 596820
Data columns (total 7 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   icb_ons_code                       596821 non-null  object
 1   appointment_month                  596821 non-null  object
 2   appointment_status                 596821 non-null  object
 3   hcp_type                           596821 non-null  object
 4   appointment_mode                   596821 non-null  object
 5   time_between_book_and_appointment  596821 non-null  object
 6   count_of_appointments              596821 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 31.9+ MB


In [13]:
# Convert the appointment month to datetime type
ar['appointment_month'] = pd.to_datetime(ar['appointment_month'])

# Determine the record period of ad dataframe
ar['appointment_month'].agg(['min','max'])

min   2020-01-01
max   2022-06-01
Name: appointment_month, dtype: datetime64[ns]

In [104]:
# Create some additional columns of dates in ad DataFrame
month_dict={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May',6:'Jun',
            7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

season_dict={1:'Winter',2:'Winter',3:'Winter',
             4:'Spring',5:'Spring',6:'Spring',
             7:'Summer',8:'Summer',9:'Summer',
             10:'Autumn',11:'Autumn',12:'Autumn'}

region_dict={'E40000003':'London', 'E40000005':'South East',
             'E40000006':'South West', 'E40000007':'East of England', 
             'E40000010':'North West', 'E40000011':'Midlands', 'E40000012':'North East and Yorkshire'}

ar['year']=ar['appointment_month'].dt.year
ar['quarter']=ar['appointment_month'].dt.quarter
ar['month']= ar['appointment_month'].dt.month
ar['month_year']=ar['month'].map(month_dict).astype(str)+'-'+ar.year.astype(str)
ar['season']=ar['month'].map(season_dict)

code = ad[['icb_ons_code','region_ons_code']].drop_duplicates()
code_dict = dict(zip(code.icb_ons_code,code.region_ons_code))
ar['region_ons_code']=ar['icb_ons_code'].map(code_dict)

region_dict={'E40000003':'London', 'E40000005':'South East',
             'E40000006':'South West', 'E40000007':'East of England', 
             'E40000010':'North West', 'E40000011':'Midlands', 'E40000012':'North East and Yorkshire'}

ar['region']=ar['region_ons_code'].map(region_dict)

ar.head()

,icb_ons_code,appointment_month,appointment_status,hcp_type,appointment_mode,time_between_book_and_appointment,count_of_appointments,year,quarter,month,month_year,season,region_ons_code,region
0,E54000034,2020-01-01,Attended,GP,Face-to-Face,1 Day,8107,2020,1,1,Jan-2020,Winter,E40000005,South East
1,E54000034,2020-01-01,Attended,GP,Face-to-Face,15 to 21 Days,6791,2020,1,1,Jan-2020,Winter,E40000005,South East
2,E54000034,2020-01-01,Attended,GP,Face-to-Face,2 to 7 Days,20686,2020,1,1,Jan-2020,Winter,E40000005,South East
3,E54000034,2020-01-01,Attended,GP,Face-to-Face,22 to 28 Days,4268,2020,1,1,Jan-2020,Winter,E40000005,South East
4,E54000034,2020-01-01,Attended,GP,Face-to-Face,8 to 14 Days,11971,2020,1,1,Jan-2020,Winter,E40000005,South East


In [16]:
# Determine the number of unique elements of the data set.
ar.nunique()

icb_ons_code                            42
appointment_month                       30
appointment_status                       3
hcp_type                                 3
appointment_mode                         5
time_between_book_and_appointment        8
count_of_appointments                22807
year                                     3
quarter                                  4
month                                   12
month_year                              30
season                                   4
region_ons_code                          7
region                                   7
dtype: int64

In [17]:
# Determine whether there are missing values.
ar.isnull().sum()

icb_ons_code                         0
appointment_month                    0
appointment_status                   0
hcp_type                             0
appointment_mode                     0
time_between_book_and_appointment    0
count_of_appointments                0
year                                 0
quarter                              0
month                                0
month_year                           0
season                               0
region_ons_code                      0
region                               0
dtype: int64

In [18]:
# Determine the number of appointment records in ar dataframe
ar['count_of_appointments'].sum()

742804525

In [19]:
# Determine the number of duplicated record
ar.duplicated().sum()

# Keep all records since there is no record of every single appointments for cross-check. 
# The total numbers of appointment match with nc dataset.

21604

In [20]:
# Determine the descriptive statistics of the data set.
ar.describe()

,count_of_appointments,year,quarter,month
count,596821.000000,596821.000000,596821.000000,596821.000000
mean,1244.601857,2020.806061,2.300403,5.903294
std,5856.887042,0.750589,1.106084,3.414846
min,1.000000,2020.000000,1.000000,1.000000
25%,7.000000,2020.000000,1.000000,3.000000
50%,47.000000,2021.000000,2.000000,6.000000
75%,308.000000,2021.000000,3.000000,9.000000
max,211265.000000,2022.000000,4.000000,12.000000


## Handle dataset 3: national_categories

In [21]:
# Import and sense-check the national_categories.xlsx data set as nc.
nc = pd.read_excel('national_categories.xlsx')

# View the DataFrame.
nc.head()

,appointment_date,icb_ons_code,sub_icb_location_name,service_setting,context_type,national_category,count_of_appointments,appointment_month
0,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,Primary Care Network,Care Related Encounter,Patient contact during Care Home Round,3,2021-08
1,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,Other,Care Related Encounter,Planned Clinics,7,2021-08
2,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,General Practice,Care Related Encounter,Home Visit,79,2021-08
3,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,General Practice,Care Related Encounter,General Consultation Acute,725,2021-08
4,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,General Practice,Care Related Encounter,Structured Medication Review,2,2021-08


In [32]:
# Determine the number of categories in each variable.
nc.nunique()

appointment_date          334
icb_ons_code               42
sub_icb_location_name     106
service_setting             5
context_type                3
national_category          18
count_of_appointments    9957
appointment_month          11
dtype: int64

In [33]:
# Determine the metadata of the data set.
nc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817394 entries, 0 to 817393
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   appointment_date       817394 non-null  datetime64[ns]
 1   icb_ons_code           817394 non-null  object        
 2   sub_icb_location_name  817394 non-null  object        
 3   service_setting        817394 non-null  object        
 4   context_type           817394 non-null  object        
 5   national_category      817394 non-null  object        
 6   count_of_appointments  817394 non-null  int64         
 7   appointment_month      817394 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 49.9+ MB


In [34]:
# Determine whether there are missing values.
nc.isnull().sum()

appointment_date         0
icb_ons_code             0
sub_icb_location_name    0
service_setting          0
context_type             0
national_category        0
count_of_appointments    0
appointment_month        0
dtype: int64

In [35]:
# Convert the appointment date to datetime type
nc['appointment_date'] = pd.to_datetime(nc['appointment_date'])

# Determine the record period of ad dataframe
nc['appointment_date'].agg(['min','max'])

min   2021-08-01
max   2022-06-30
Name: appointment_date, dtype: datetime64[ns]

In [105]:
# Create some additional columns of dates in nc DataFrame

month_dict={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May',6:'Jun',
            7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

season_dict={1:'Winter',2:'Winter',3:'Winter',
             4:'Spring',5:'Spring',6:'Spring',
             7:'Summer',8:'Summer',9:'Summer',
             10:'Autumn',11:'Autumn',12:'Autumn'}

nc['year']=nc['appointment_date'].dt.year
nc['quarter']=nc['appointment_date'].dt.quarter
nc['month']=nc['appointment_date'].dt.month
nc['month_year']=nc['month'].map(month_dict).astype(str)+'-'+nc.year.astype(str)
nc['season']=nc['month'].map(season_dict)
nc['weekday']=nc['appointment_date'].dt.day_name()

code = ad[['icb_ons_code','region_ons_code']].drop_duplicates()
code_dict = dict(zip(code.icb_ons_code,code.region_ons_code))
nc['region_ons_code']=nc['icb_ons_code'].map(code_dict)


region_dict={'E40000003':'London', 'E40000005':'South East',
             'E40000006':'South West', 'E40000007':'East of England', 
             'E40000010':'North West', 'E40000011':'Midlands', 'E40000012':'North East and Yorkshire'}

nc['region']=nc['region_ons_code'].map(region_dict)

nc.head()

,appointment_date,icb_ons_code,sub_icb_location_name,service_setting,context_type,national_category,count_of_appointments,appointment_month,year,quarter,month,month_year,season,weekday,region_ons_code,region
0,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,Primary Care Network,Care Related Encounter,Patient contact during Care Home Round,3,2021-08,2021,3,8,Aug-2021,Summer,Monday,E40000012,North East and Yorkshire
1,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,Other,Care Related Encounter,Planned Clinics,7,2021-08,2021,3,8,Aug-2021,Summer,Monday,E40000012,North East and Yorkshire
2,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,General Practice,Care Related Encounter,Home Visit,79,2021-08,2021,3,8,Aug-2021,Summer,Monday,E40000012,North East and Yorkshire
3,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,General Practice,Care Related Encounter,General Consultation Acute,725,2021-08,2021,3,8,Aug-2021,Summer,Monday,E40000012,North East and Yorkshire
4,2021-08-02,E54000050,NHS North East and North Cumbria ICB - 00L,General Practice,Care Related Encounter,Structured Medication Review,2,2021-08,2021,3,8,Aug-2021,Summer,Monday,E40000012,North East and Yorkshire


In [37]:
# Determine the number of duplicated record
nc.duplicated().sum()

0

In [38]:
# Determine the descriptive statistics of the data set.
nc.describe()

,count_of_appointments,year,quarter,month
count,817394.000000,817394.000000,817394.000000,817394.000000
mean,362.183684,2021.548196,2.449228,6.456048
std,1084.576600,0.497672,1.162157,3.596845
min,1.000000,2021.000000,1.000000,1.000000
25%,7.000000,2021.000000,1.000000,3.000000
50%,25.000000,2022.000000,2.000000,6.000000
75%,128.000000,2022.000000,4.000000,10.000000
max,16590.000000,2022.000000,4.000000,12.000000


# Investigate staffing issue 

### Appointment VS Capacity

In [295]:
# Sum the appointments: by month

# Calculate total appointments per month (Jan 2020 - Jun 2022)
ar_df = ar.groupby(['appointment_month','appointment_status'])['count_of_appointments'].agg('sum').reset_index()

# Calculate the calendars of month
ar_df['daysinmonth'] = ar_df['appointment_month'].apply(lambda t: pd.Period(t, freq='S').days_in_month)

# Calculate the monthly max capacity
ar_df['month_capacity']=1200000*ar_df['daysinmonth']

# Calculate the expected capacity utilisation rate
ar_df['expected_utilisation_rate'] = ar_df['count_of_appointments']/ar_df['month_capacity']*100

# Calculate the actual capacity utilisation for attended appointments
def actual_utilisation(row):
   if row['appointment_status'] == 'Attended':
        return row['count_of_appointments']
ar_df['actual_utilisation'] = ar_df.apply(actual_utilisation, axis=1)
ar_df['actual_utilisation'].fillna(0, inplace=True)

# Calculate the actual capacity utilisation rate
def actual_utilisation_rate(row):
    if row['appointment_status'] == 'Attended':
        return row['expected_utilisation_rate']

ar_df['actual_utilisation_rate'] = ar_df.apply(actual_utilisation_rate, axis=1)
ar_df['actual_utilisation_rate'].fillna(0, inplace=True)

ar_df_line = ar_df.groupby(['appointment_month'])['expected_utilisation_rate','actual_utilisation_rate'].agg('sum').reset_index()
fig = px.line(ar_df_line, x='appointment_month', y=ar_df_line.columns[1:],
              template='none', title='Capacity utilization rate per month',
              labels={'variable':'Utilization Rate',
                      'value': 'Percentage against the maximum capacity',
                     'appointment_month': 'Appointment Month'})
fig.update_xaxes(dtick="M1", tickangle=45)
fig.add_hline(y=100,line_width=3, line_dash="dash", line_color='green', name='Capacity')
fig.update_yaxes(range=[0, 100])
fig.show()

In [294]:
#Total appointments (ar)

data = ar.groupby(['appointment_month','season'])['count_of_appointments'].agg('sum').reset_index()

fig = px.bar(data, x='appointment_month', y='count_of_appointments', 
             color='season',
             category_orders={'season': ['Spring', 'Summer', 'Autumn', 'Winter']},
             text='count_of_appointments', template='none')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_layout(title = 'Number of appointments per month', 
                  xaxis_title = "Month", yaxis_title = "Number of Appointments")
fig.show()

# Regions

In [363]:
data = ar.groupby(['appointment_month','region'])['count_of_appointments'].agg('sum').reset_index()

fig = px.line(data, x='appointment_month', y='count_of_appointments',
             template='none', color='region',width=800, height=400)
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_yaxes(rangemode="tozero")
fig.update_layout(title = 'Number of appointments per month for region', 
                  xaxis_title = "Month", yaxis_title = "Numbers of appointments")
fig.show()

In [357]:
# 100% stack bar plots

data = ar.groupby(['region','appointment_status'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby('region')['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='region', y='Percentage',color='appointment_status',
             category_orders={'region': ['Midlands', 'North East and Yorkshire',
                                         'South East', 'London', 'North West',
                                         'East of England','South West']},
                              title="Appointment per region", template='none', text_auto= True)   

fig.show()

## HCP types

In [297]:
data = ar.groupby(['appointment_month','hcp_type'])['count_of_appointments'].agg('sum').reset_index()

fig = px.pie(data, values='count_of_appointments', 
             names='hcp_type', template='none')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_layout(title = 'Appointments by hcp types' ,
                  xaxis_title = "Month", yaxis_title = "Number of Appointments")
fig.show()

In [298]:
data = ar.groupby(['appointment_month','hcp_type'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby('appointment_month')['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='appointment_month', y='Percentage',color='hcp_type',
        title='Appointments per hcp type', template='none', text_auto= True)
fig.update_xaxes(dtick="M1", tickangle=45)
fig.show()

In [315]:
data = ar.groupby(['hcp_type','appointment_status'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby('hcp_type')['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='hcp_type', y='Percentage',color='appointment_status',
        title='Appointments per hcp type', template='none', text_auto= True)   
fig.update_layout(barmode='group')
fig.show()

In [299]:
data = ar.groupby(['appointment_month','hcp_type'])['count_of_appointments'].agg('sum').reset_index()

fig = px.line(data, x='appointment_month', y='count_of_appointments',
             template='none', color='hcp_type',width=800, height=400)
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_yaxes(rangemode="tozero")
fig.update_layout(title = 'Number of appointments per month for hcp type', 
                  xaxis_title = "Month", yaxis_title = "Numbers of appointments")
fig.show()

## Appointment modes

In [300]:
# 1.3 Total appointments (ar) - mode
# 1.3.1 Pie chart - mode % 
data = ar.groupby(['appointment_month','appointment_mode'])['count_of_appointments'].agg('sum').reset_index()

fig = px.pie(data, values='count_of_appointments', 
             names='appointment_mode', template='none',
             category_orders=
             {'appointment_mode':['Face-to-Face', 'Telephone','Home Visit', 'Video/Online','Unknown']})
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_layout(title = 'Appointments by appointment modes', 
                  xaxis_title = "Month", yaxis_title = "Number of Appointments")
fig.show()

In [301]:
data = ar.groupby(['appointment_month','appointment_mode'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby('appointment_month')['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='appointment_month', y='Percentage',color='appointment_mode',
        title='Appointments per mode', template='none', text_auto= True,category_orders=
             {'appointment_mode':['Face-to-Face', 'Telephone','Home Visit', 'Video/Online','Unknown']})
#fig.update_layout(barmode="relative")
fig.update_xaxes(dtick="M1", tickangle=45)
fig.show()

In [302]:
data = ar.groupby(['appointment_month','appointment_mode'])['count_of_appointments'].agg('sum').reset_index()

fig = px.line(data, x='appointment_month', y='count_of_appointments',
             template='none', color='appointment_mode',width=800, height=400,
              category_orders=
             {'appointment_mode':['Face-to-Face', 'Telephone','Home Visit', 'Video/Online','Unknown']})
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_yaxes(rangemode="tozero")
fig.update_layout(title = 'Number of appointments per month for appointment mode', 
                  xaxis_title = "Month", yaxis_title = "Numbers of appointments")
fig.show()

## Waiting time between booking and appointment

In [303]:
data = ar.groupby(['appointment_month','time_between_book_and_appointment'])['count_of_appointments'].agg('sum').reset_index()

fig = px.pie(data, values='count_of_appointments', 
             names='time_between_book_and_appointment', 
             category_orders={'time_between_book_and_appointment': 
                              ['Same Day','1 Day', '2 to 7 Days','8  to 14 Days','15  to 21 Days', 
                               '22  to 28 Days', 'More than 28 Days','Unknown / Data Quality']},
             template='none')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_layout(title = 'Number of appointments per waiting time', 
                  xaxis_title = "Month", yaxis_title = "Number of Appointments")
fig.show()

In [304]:
data = ar.groupby(['appointment_month','time_between_book_and_appointment'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby('appointment_month')['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='appointment_month', y='Percentage',color='time_between_book_and_appointment',
        title='Appointments per waiting time', template='none', text_auto= True,                           
             category_orders={'time_between_book_and_appointment': 
                              ['Same Day','1 Day', '2 to 7 Days','8  to 14 Days','15  to 21 Days', 
                               '22  to 28 Days', 'More than 28 Days','Unknown / Data Quality']})
fig.update_xaxes(dtick="M1", tickangle=45)
fig.show()

In [305]:
data = ar.groupby(['appointment_month','time_between_book_and_appointment'])['count_of_appointments'].agg('sum').reset_index()

fig = px.line(data, x='appointment_month', y='count_of_appointments',
             template='none', color='time_between_book_and_appointment',width=1000, height=400,
                           category_orders={'time_between_book_and_appointment': 
                              ['Same Day','1 Day', '2 to 7 Days','8  to 14 Days','15  to 21 Days', 
                               '22  to 28 Days', 'More than 28 Days','Unknown / Data Quality']})
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_yaxes(rangemode="tozero")
fig.update_layout(title = 'Number of appointments per month for waiting time', 
                  xaxis_title = "Month", yaxis_title = "Numbers of appointments")
fig.show()

## Appointment Status

In [307]:
data = ar.groupby(['appointment_month','appointment_status'])['count_of_appointments'].agg('sum').reset_index()

fig = px.line(data, x='appointment_month', y='count_of_appointments',
             template='none', color='appointment_status',width=1000, height=400)
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_yaxes(rangemode="tozero")
fig.update_layout(title = 'Number of appointments per appointment status', 
                  xaxis_title = "Month", yaxis_title = "Numbers of appointments")
fig.show()

In [308]:
data = ar.groupby(['appointment_month','appointment_status','season'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby('appointment_month')['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='appointment_month', y='Percentage', color='season',
             category_orders={'season': ['Spring', 'Summer', 'Autumn', 'Winter']},
             facet_row='appointment_status',
            title='Total appointments per status', template='none', text_auto= True)   

fig.update_xaxes(dtick="M1", tickangle=45)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.show()

# Patient pathway

In [280]:


data = ar.groupby(['month_year','hcp_type','appointment_mode','appointment_status'])['count_of_appointments'].agg('sum').reset_index()

px.bar(data, x='appointment_mode', y='count_of_appointments',color='appointment_status',
       facet_col ='hcp_type',
       title="Number of appointment per hcp types and appointment mode", template='none', height=400)

In [260]:
data = ar.groupby(['hcp_type','appointment_mode','appointment_status','time_between_book_and_appointment'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby(['hcp_type','appointment_mode','time_between_book_and_appointment'])['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='appointment_mode', y='Percentage',color='appointment_status',
             facet_row='time_between_book_and_appointment', animation_frame = 'hcp_type',
             category_orders=
             {'time_between_book_and_appointment': 
              ['Same Day','1 Day', '2 to 7 Days','8  to 14 Days','15  to 21 Days', 
               '22  to 28 Days', 'More than 28 Days','Unknown / Data Quality'],
              'appointment_mode':['Face-to-Face', 'Telephone','Home Visit', 'Video/Online','Unknown']},
             title='Appointment attendance % per hcp, mode, waiting time', template='none', text_auto= True,
             width=800, height=1500)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.update_xaxes(dtick="M1", tickangle=45)

fig.show()

In [325]:
data = ar.groupby(['hcp_type','appointment_mode','appointment_status','time_between_book_and_appointment'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby(['hcp_type','appointment_mode','time_between_book_and_appointment'])['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='appointment_mode', y='Percentage',color='appointment_status',
             facet_row='time_between_book_and_appointment', animation_frame = 'hcp_type',
             category_orders=
             {'time_between_book_and_appointment': 
              ['Same Day','1 Day', '2 to 7 Days','8  to 14 Days','15  to 21 Days', 
               '22  to 28 Days', 'More than 28 Days','Unknown / Data Quality'],
              'appointment_mode':['Face-to-Face', 'Telephone','Home Visit', 'Video/Online','Unknown']},
             title='Appointment attendance % per hcp, mode, waiting time', template='none', text_auto= True,
             width=800, height=1500)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.update_xaxes(dtick="M1", tickangle=45)

fig.show()

In [309]:
data = ar.groupby(['region','hcp_type','appointment_mode','appointment_status','time_between_book_and_appointment'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby(['region','hcp_type','appointment_mode','time_between_book_and_appointment'])['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='appointment_mode', y='Percentage',color='appointment_status',
             facet_col='hcp_type',
             facet_row='time_between_book_and_appointment', animation_frame = 'region',
             category_orders=
             {'time_between_book_and_appointment': 
              ['Same Day','1 Day', '2 to 7 Days','8  to 14 Days','15  to 21 Days', 
               '22  to 28 Days', 'More than 28 Days','Unknown / Data Quality'],
              'appointment_mode':['Face-to-Face', 'Telephone','Home Visit', 'Video/Online','Unknown']},
             title='Appointment attendance % per region', template='none', text_auto= True,
             width=1500, height=1500)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.update_xaxes(dtick="M1", tickangle=45)

fig.show()

In [364]:

data = ar.groupby(['month_year','hcp_type','appointment_mode','time_between_book_and_appointment','appointment_status'])['count_of_appointments'].agg('sum').reset_index()

fig = px.bar(data, x='appointment_mode', y='count_of_appointments',color='appointment_status',
       facet_row ='time_between_book_and_appointment', animation_frame='hcp_type',
       category_orders={'time_between_book_and_appointment': 
                        ['Same Day','1 Day', '2 to 7 Days','8  to 14 Days','15  to 21 Days', 
                         '22  to 28 Days', 'More than 28 Days','Unknown / Data Quality']},
                        title="Number of Appointments", template='none', height=1900)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

# Actual Duration

In [365]:
data = ad.groupby(['actual_duration','month_year'])['count_of_appointments'].agg('sum').reset_index()

fig = px.bar(data, x='month_year', y='count_of_appointments',
              category_orders={'month_year': 
                        ['Dec-2021', 'Jan-2022', 'Feb-2022', 'Mar-2022', 'Apr-2022',
                         'May-2022', 'Jun-2022'],
                               'actual_duration':['1-5 Minutes','6-10 Minutes','11-15 Minutes',
                                                  '16-20 Minutes','21-30 Minutes','31-60 Minutes'
                                                  'Unknown / Data Quality']},
             template='none', color='actual_duration',width=1000, height=400)
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_yaxes(rangemode="tozero")
fig.update_layout(title = 'Number of appointments per actual duration', 
                  xaxis_title = "Month", yaxis_title = "Numbers of appointments")
fig.show()

In [322]:
# Appointment Distribution [HCP type, Appointment mode, Waiting time, Final Status]

data = ad.groupby(['actual_duration','region'])['count_of_appointments'].sum().reset_index()

fig =px.sunburst(data, 
                 path=['actual_duration','region'],
                 values='count_of_appointments',
                       title="Appointment duartion per region",
                  width=750, height=750, template = 'none')
fig.update_traces(textinfo="label+percent parent")
fig.show()

## National Category

In [331]:
data = nc.groupby(['national_category','appointment_month'])['count_of_appointments'].agg('sum').reset_index()
data['Percentage'] = 100 * data['count_of_appointments'] / data.groupby('appointment_month')['count_of_appointments'].transform('sum')
data['Percentage'] = data['Percentage'].apply(lambda x: '{0:.2f}%'.format(x))

fig = px.bar(data, x='appointment_month', y='Percentage',color='national_category',
        title='Appointment per national category', template='none', text_auto= True)                           
fig.update_xaxes(dtick="M1", tickangle=45)
fig.show()

In [332]:
data = nc.groupby(['national_category','appointment_month'])['count_of_appointments'].agg('sum').reset_index()

fig = px.line(data, x='appointment_month', y='count_of_appointments',
             template='none', color='national_category',width=1000, height=400)
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_yaxes(rangemode="tozero")
fig.update_layout(title = 'Number of appointments per national category', 
                  xaxis_title = "Month", yaxis_title = "Numbers of appointments")
fig.show()

In [326]:
# Appointment per national category

data = nc.groupby(['service_setting','national_category'])['count_of_appointments'].agg('sum').reset_index()
new_data = data.sort_values('count_of_appointments',ascending = False)

fig = px.bar(new_data, x='national_category', y='count_of_appointments', 
             color='service_setting',
             text='count_of_appointments', template='none')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_traces(opacity=0.85)
fig.update_xaxes(dtick="M1", tickangle=45)
fig.update_layout(title = 'Appointment per national category', 
                  xaxis_title = 'National Category', yaxis_title = "Number of Appointments")
fig.show()

In [324]:
# Appointment Distribution per region,service_setting,national_category

data = nc.groupby(['service_setting','national_category'])['count_of_appointments'].sum().reset_index()

fig =px.sunburst(data, 
                 path=['service_setting','national_category'],
                 values='count_of_appointments',
                       title="Total Appointment Distribution [region,service_setting,national_category]",
                  width=750, height=750,template = 'none')
fig.update_traces(textinfo="label+percent parent")
fig.show()

In [323]:
# Appointment Distribution [region,service_setting,national_category]

data = nc.groupby(['region','service_setting','national_category'])['count_of_appointments'].sum().reset_index()

fig =px.sunburst(data, 
                 path=['region','service_setting','national_category'],
                 values='count_of_appointments',
                       title="Total Appointment Distribution [region,service_setting,national_category]",
                  width=750, height=750,template = 'none')
fig.update_traces(textinfo="label+percent parent")
fig.show()

# No Show Appointment

In [311]:
# DNA appointments distribution (patient pathway)

# Appointment Distribution [HCP type, Appointment mode, Waiting time, Final Status]
data = ar.groupby(['appointment_month','region','hcp_type','appointment_mode','time_between_book_and_appointment','appointment_status'])['count_of_appointments'].sum().reset_index()
filter = (data['appointment_status']=='DNA') & (data['appointment_month']=='2022-04-01')

fig =px.sunburst(data[filter], 
                 path=['region','hcp_type','appointment_mode'], #'time_between_book_and_appointment'],
                 values='count_of_appointments',
                 title='DNA Appointment Distribution - region, HCP type, appointment mode, Waiting time',
                  width=750, height=750)
fig.update_traces(textinfo="label+percent parent")
fig.show()

In [358]:
data = ar[ar['appointment_status'] == 'DNA']
data = data.groupby(['appointment_status','hcp_type','appointment_mode','time_between_book_and_appointment'])['count_of_appointments'].agg('sum').reset_index()

fig = px.bar(data, x='appointment_mode', y='count_of_appointments',color='time_between_book_and_appointment',
             facet_col ='hcp_type',facet_row ='appointment_status',
             category_orders={'time_between_book_and_appointment': 
                              ['Same Day','1 Day', '2 to 7 Days','8  to 14 Days','15  to 21 Days', 
                               '22  to 28 Days', 'More than 28 Days','Unknown / Data Quality']},
             title="DNA Appointments", template='none')   
fig.update_layout(barmode="relative")
fig.show()

In [359]:
# Tree plot
# Determine the trend of DNA appintment.
ar_filter = (ar['appointment_status'] == "DNA")

# View the output.
filtered_ar = ar.loc[ar_filter]

ar_tree = filtered_ar.groupby(['hcp_type','appointment_mode','time_between_book_and_appointment'])['count_of_appointments'].sum().reset_index()

ar_tree["all"] = "all" # in order to have a single root node
fig = px.treemap(ar_tree, 
                 path=['all','hcp_type','appointment_mode','time_between_book_and_appointment'], 
                 values='count_of_appointments', title="DNA appointments per month, appointment mode, waiting time") # 面积大小用total_bill字段决定
                
fig.update_traces(textinfo="label+percent parent")
fig.show()

In [360]:
# DNA Appointment Distribution [HCP type, Appointment mode, Waiting time, Final Status]

data = ar.groupby(['hcp_type','appointment_mode','time_between_book_and_appointment','appointment_status'])['count_of_appointments'].sum().reset_index()
filter = data['appointment_status']=='DNA'

fig =px.sunburst(data[filter], 
                 path=['hcp_type','appointment_mode','time_between_book_and_appointment'],
                 values='count_of_appointments',
                       title='DNA Appointment Distribution - region, HCP type, appointment mode, Waiting time',
                  width=750, height=750,template = 'none')
fig.update_traces(textinfo="label+percent parent")
fig.show()